In [99]:
import pyodbc
import json
import requests

In [100]:
pyodbc.dataSources()

{'dBASE Files': 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'Excel Files': 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'MS Access Database': 'Microsoft Access Driver (*.mdb, *.accdb)'}

In [101]:
driver =  '{Microsoft Access Driver (*.mdb, *.accdb)}'
filePath = r'C:\Users\Danyal\Desktop\New folder\HanvonFaceIDManager1.mdb'

In [102]:
myDataSources = pyodbc.dataSources()
access_driver = myDataSources[ 'MS Access Database']

In [103]:
connection = pyodbc.connect(driver = access_driver,dbq = filePath, autocommit=True)

In [104]:
cursor= connection.cursor()

In [105]:
details = []
tables = list(cursor.tables())
for table in tables:
    details.append(table[2])

In [106]:
details

['MSysAccessObjects',
 'MSysAccessXML',
 'MSysACEs',
 'MSysNavPaneGroupCategories',
 'MSysNavPaneGroups',
 'MSysNavPaneGroupToObjects',
 'MSysNavPaneObjectIDs',
 'MSysObjects',
 'MSysQueries',
 'MSysRelationships',
 'AdminPrivileges',
 'Attendance',
 'Audit',
 'AuditType',
 'CompanyType',
 'Department',
 'Employee',
 'Employee_Location',
 'EmployeeUpdateDetail',
 'FormRights',
 'Forms',
 'HRTCompanyLogic',
 'Location',
 'Machine',
 'Machine_TimeStatus',
 'MachineType',
 'OperationLog',
 'Paste Errors',
 'UserRightsOnForm',
 'Users',
 'VerificationModes',
 'Audit_Details_View',
 'Employee_Detail_View',
 'Employee_Location_Second_View',
 'Employee_Machine_View',
 'Employee_Search_Detail_View',
 'Machine_Detail_View',
 'Query2',
 'User_Form_Access_View']

In [107]:
# defining query
tableName = 'Attendance'
query = 'select * from {}'.format(tableName)
cursor.execute(query)

In [108]:
#fetching all rows
rows = cursor.fetchall()
rows

[(1, 1, '289', '09-Dec-2017 08:24:53 PM', '01', 'Test', '09-Dec-2017', '08:24 PM', None, '11-Dec-2017 15:03:36 PM'),
 (2, 1, '158', '09-Dec-2017 07:46:33 PM', '01', 'Test', '09-Dec-2017', '07:46 PM', None, '11-Dec-2017 15:03:36 PM'),
 (3, 1, '2470', '08-Dec-2017 07:54:15 PM', '02', 'Test', '08-Dec-2017', '07:54 PM', None, '11-Dec-2017 15:03:36 PM'),
 (4, 1, '375', '11-Dec-2017 08:07:12 AM', '02', 'Test', '11-Dec-2017', '08:07 AM', None, '11-Dec-2017 15:03:37 PM'),
 (5, 1, '2614', '08-Dec-2017 07:33:19 PM', '01', 'Test', '08-Dec-2017', '07:33 PM', None, '11-Dec-2017 15:03:37 PM'),
 (6, 1, '2194', '11-Dec-2017 07:45:14 AM', '02', 'Test', '11-Dec-2017', '07:45 AM', None, '11-Dec-2017 15:03:37 PM'),
 (7, 1, '625', '09-Dec-2017 08:09:10 AM', '01', 'Test', '09-Dec-2017', '08:09 AM', None, '11-Dec-2017 15:03:38 PM'),
 (8, 1, '2634', '07-Dec-2017 08:08:48 AM', '02', 'Test', '07-Dec-2017', '08:08 AM', None, '11-Dec-2017 15:03:38 PM'),
 (9, 1, '228', '08-Dec-2017 08:06:21 AM', '01', 'Test', '08-

In [109]:
len(rows)

1996

In [111]:
data = {'items':[]} #Converting data into Json
with open('names.json', 'w') as f:
    for row in rows:
        data['items'].append({'mchdb_pk': row[0],'sensorid':row[1],'userid': row[2],'checktime': row[3],'verifycode':row[4],'checktype':row[5]})

In [121]:
for x in data['items']:
    if x['sensorid'] == 1:
        print(x)

{'mchdb_pk': 1, 'sensorid': 1, 'userid': '289', 'checktime': '09-Dec-2017 08:24:53 PM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 2, 'sensorid': 1, 'userid': '158', 'checktime': '09-Dec-2017 07:46:33 PM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 3, 'sensorid': 1, 'userid': '2470', 'checktime': '08-Dec-2017 07:54:15 PM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 4, 'sensorid': 1, 'userid': '375', 'checktime': '11-Dec-2017 08:07:12 AM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 5, 'sensorid': 1, 'userid': '2614', 'checktime': '08-Dec-2017 07:33:19 PM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 6, 'sensorid': 1, 'userid': '2194', 'checktime': '11-Dec-2017 07:45:14 AM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 7, 'sensorid': 1, 'userid': '625', 'checktime': '09-Dec-2017 08:09:10 AM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 8, 'sensorid': 1, 'userid': '2634', 'checktime': '07-Dec-2017 08:08:48 AM', 'veri

{'mchdb_pk': 1173, 'sensorid': 1, 'userid': '2245', 'checktime': '10-Dec-2017 08:05:08 AM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 1174, 'sensorid': 1, 'userid': '2182', 'checktime': '09-Dec-2017 07:41:22 PM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 1175, 'sensorid': 1, 'userid': '2864', 'checktime': '09-Dec-2017 07:52:20 PM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 1176, 'sensorid': 1, 'userid': '19', 'checktime': '07-Dec-2017 05:54:22 AM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 1177, 'sensorid': 1, 'userid': '2587', 'checktime': '07-Dec-2017 08:33:41 PM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 1178, 'sensorid': 1, 'userid': '3216', 'checktime': '09-Dec-2017 01:56:08 PM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 1179, 'sensorid': 1, 'userid': '696', 'checktime': '06-Dec-2017 07:57:10 PM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 1180, 'sensorid': 1, 'userid': '184', 'checktime': '07-Dec-

In [142]:

for index, value in enumerate(data['items']):
    r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att', json = value)

In [129]:
for y in data['items']: #sending data to their location
     if y['verifycode'] == '02':
            r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=2',json = y)
            print(y)

for z in data['items']: #sending data to their location
    if z['verifycode'] == '03':
        r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=3',json = z)

{'mchdb_pk': 3, 'sensorid': 1, 'userid': '2470', 'checktime': '08-Dec-2017 07:54:15 PM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 4, 'sensorid': 1, 'userid': '375', 'checktime': '11-Dec-2017 08:07:12 AM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 6, 'sensorid': 1, 'userid': '2194', 'checktime': '11-Dec-2017 07:45:14 AM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 8, 'sensorid': 1, 'userid': '2634', 'checktime': '07-Dec-2017 08:08:48 AM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 11, 'sensorid': 1, 'userid': '585', 'checktime': '09-Dec-2017 08:21:21 AM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 14, 'sensorid': 1, 'userid': '625', 'checktime': '10-Dec-2017 07:46:27 PM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 15, 'sensorid': 1, 'userid': '3174', 'checktime': '09-Dec-2017 07:48:15 PM', 'verifycode': '02', 'checktype': 'Test'}
{'mchdb_pk': 16, 'sensorid': 1, 'userid': '3241', 'checktime': '09-Dec-2017 08:17:47 PM', 

KeyboardInterrupt: 

0: {'mchdb_pk': 1, 'sensorid': 1, 'userid': '289', 'checktime': '09-Dec-2017 08:24:53 PM', 'verifycode': '01', 'checktype': 'Test'}
1: {'mchdb_pk': 2, 'sensorid': 1, 'userid': '158', 'checktime': '09-Dec-2017 07:46:33 PM', 'verifycode': '01', 'checktype': 'Test'}
2: {'mchdb_pk': 3, 'sensorid': 1, 'userid': '2470', 'checktime': '08-Dec-2017 07:54:15 PM', 'verifycode': '02', 'checktype': 'Test'}
3: {'mchdb_pk': 4, 'sensorid': 1, 'userid': '375', 'checktime': '11-Dec-2017 08:07:12 AM', 'verifycode': '02', 'checktype': 'Test'}
4: {'mchdb_pk': 5, 'sensorid': 1, 'userid': '2614', 'checktime': '08-Dec-2017 07:33:19 PM', 'verifycode': '01', 'checktype': 'Test'}
5: {'mchdb_pk': 6, 'sensorid': 1, 'userid': '2194', 'checktime': '11-Dec-2017 07:45:14 AM', 'verifycode': '02', 'checktype': 'Test'}
6: {'mchdb_pk': 7, 'sensorid': 1, 'userid': '625', 'checktime': '09-Dec-2017 08:09:10 AM', 'verifycode': '01', 'checktype': 'Test'}
7: {'mchdb_pk': 8, 'sensorid': 1, 'userid': '2634', 'checktime': '07-Dec-

647: {'mchdb_pk': 652, 'sensorid': 1, 'userid': '2643', 'checktime': '11-Dec-2017 08:09:14 AM', 'verifycode': '01', 'checktype': 'Test'}
648: {'mchdb_pk': 653, 'sensorid': 1, 'userid': '2599', 'checktime': '07-Dec-2017 07:59:15 AM', 'verifycode': '01', 'checktype': 'Test'}
649: {'mchdb_pk': 654, 'sensorid': 1, 'userid': '1823', 'checktime': '08-Dec-2017 07:57:27 AM', 'verifycode': '02', 'checktype': 'Test'}
650: {'mchdb_pk': 655, 'sensorid': 1, 'userid': '68', 'checktime': '10-Dec-2017 07:31:30 PM', 'verifycode': '01', 'checktype': 'Test'}
651: {'mchdb_pk': 656, 'sensorid': 1, 'userid': '998', 'checktime': '06-Dec-2017 07:52:37 PM', 'verifycode': '02', 'checktype': 'Test'}
652: {'mchdb_pk': 657, 'sensorid': 1, 'userid': '313', 'checktime': '07-Dec-2017 07:51:25 PM', 'verifycode': '02', 'checktype': 'Test'}
653: {'mchdb_pk': 658, 'sensorid': 1, 'userid': '2598', 'checktime': '10-Dec-2017 07:56:35 AM', 'verifycode': '01', 'checktype': 'Test'}
654: {'mchdb_pk': 659, 'sensorid': 1, 'userid

1096: {'mchdb_pk': 1101, 'sensorid': 1, 'userid': '1070', 'checktime': '10-Dec-2017 09:55:42 PM', 'verifycode': '02', 'checktype': 'Test'}
1097: {'mchdb_pk': 1102, 'sensorid': 1, 'userid': '2515', 'checktime': '07-Dec-2017 03:53:11 PM', 'verifycode': '02', 'checktype': 'Test'}
1098: {'mchdb_pk': 1103, 'sensorid': 1, 'userid': '226', 'checktime': '08-Dec-2017 04:01:12 PM', 'verifycode': '02', 'checktype': 'Test'}
1099: {'mchdb_pk': 1104, 'sensorid': 1, 'userid': '1000', 'checktime': '11-Dec-2017 07:35:00 AM', 'verifycode': '01', 'checktype': 'Test'}
1100: {'mchdb_pk': 1105, 'sensorid': 1, 'userid': '3239', 'checktime': '09-Dec-2017 07:56:50 AM', 'verifycode': '01', 'checktype': 'Test'}
1101: {'mchdb_pk': 1106, 'sensorid': 1, 'userid': '791', 'checktime': '09-Dec-2017 07:46:49 PM', 'verifycode': '02', 'checktype': 'Test'}
1102: {'mchdb_pk': 1107, 'sensorid': 1, 'userid': '2900', 'checktime': '09-Dec-2017 08:33:20 AM', 'verifycode': '01', 'checktype': 'Test'}
1103: {'mchdb_pk': 1108, 'sen

1205: {'mchdb_pk': 1210, 'sensorid': 1, 'userid': '6000', 'checktime': '10-Dec-2017 04:51:25 PM', 'verifycode': '02', 'checktype': 'Test'}
1206: {'mchdb_pk': 1211, 'sensorid': 1, 'userid': '802', 'checktime': '06-Dec-2017 10:18:17 AM', 'verifycode': '01', 'checktype': 'Test'}
1207: {'mchdb_pk': 1212, 'sensorid': 1, 'userid': '2581', 'checktime': '07-Dec-2017 07:47:11 AM', 'verifycode': '01', 'checktype': 'Test'}
1208: {'mchdb_pk': 1213, 'sensorid': 1, 'userid': '2470', 'checktime': '08-Dec-2017 07:43:35 AM', 'verifycode': '01', 'checktype': 'Test'}
1209: {'mchdb_pk': 1214, 'sensorid': 1, 'userid': '2404', 'checktime': '10-Dec-2017 03:48:48 PM', 'verifycode': '01', 'checktype': 'Test'}
1210: {'mchdb_pk': 1215, 'sensorid': 1, 'userid': '2232', 'checktime': '09-Dec-2017 08:09:10 PM', 'verifycode': '02', 'checktype': 'Test'}
1211: {'mchdb_pk': 1216, 'sensorid': 1, 'userid': '2114', 'checktime': '06-Dec-2017 07:51:36 PM', 'verifycode': '01', 'checktype': 'Test'}
1212: {'mchdb_pk': 1217, 'se

1403: {'mchdb_pk': 1408, 'sensorid': 1, 'userid': '38', 'checktime': '11-Dec-2017 07:36:29 AM', 'verifycode': '01', 'checktype': 'Test'}
1404: {'mchdb_pk': 1409, 'sensorid': 1, 'userid': '2824', 'checktime': '09-Dec-2017 07:44:01 AM', 'verifycode': '01', 'checktype': 'Test'}
1405: {'mchdb_pk': 1410, 'sensorid': 1, 'userid': '2598', 'checktime': '07-Dec-2017 08:33:58 AM', 'verifycode': '02', 'checktype': 'Test'}
1406: {'mchdb_pk': 1411, 'sensorid': 1, 'userid': '60', 'checktime': '08-Dec-2017 08:33:19 AM', 'verifycode': '01', 'checktype': 'Test'}
1407: {'mchdb_pk': 1412, 'sensorid': 1, 'userid': '957', 'checktime': '10-Dec-2017 08:22:11 PM', 'verifycode': '02', 'checktype': 'Test'}
1408: {'mchdb_pk': 1413, 'sensorid': 1, 'userid': '3255', 'checktime': '07-Dec-2017 07:48:58 AM', 'verifycode': '01', 'checktype': 'Test'}
1409: {'mchdb_pk': 1414, 'sensorid': 1, 'userid': '3196', 'checktime': '08-Dec-2017 07:47:21 AM', 'verifycode': '02', 'checktype': 'Test'}
1410: {'mchdb_pk': 1415, 'sensor

1656: {'mchdb_pk': 1661, 'sensorid': 1, 'userid': '273', 'checktime': '10-Dec-2017 08:14:49 AM', 'verifycode': '01', 'checktype': 'Test'}
1657: {'mchdb_pk': 1662, 'sensorid': 1, 'userid': '2870', 'checktime': '09-Dec-2017 07:46:50 AM', 'verifycode': '01', 'checktype': 'Test'}
1658: {'mchdb_pk': 1663, 'sensorid': 1, 'userid': '625', 'checktime': '09-Dec-2017 07:45:45 PM', 'verifycode': '02', 'checktype': 'Test'}
1659: {'mchdb_pk': 1664, 'sensorid': 1, 'userid': '2867', 'checktime': '06-Dec-2017 07:54:49 PM', 'verifycode': '02', 'checktype': 'Test'}
1660: {'mchdb_pk': 1665, 'sensorid': 1, 'userid': '2542', 'checktime': '07-Dec-2017 07:54:06 PM', 'verifycode': '02', 'checktype': 'Test'}
1661: {'mchdb_pk': 1666, 'sensorid': 1, 'userid': '3281', 'checktime': '10-Dec-2017 07:59:50 AM', 'verifycode': '02', 'checktype': 'Test'}
1662: {'mchdb_pk': 1667, 'sensorid': 1, 'userid': '1013', 'checktime': '07-Dec-2017 08:18:48 AM', 'verifycode': '01', 'checktype': 'Test'}
1663: {'mchdb_pk': 1668, 'sen

1809: {'mchdb_pk': 1814, 'sensorid': 1, 'userid': '2529', 'checktime': '06-Dec-2017 07:56:13 PM', 'verifycode': '01', 'checktype': 'Test'}
1810: {'mchdb_pk': 1815, 'sensorid': 1, 'userid': '2108', 'checktime': '07-Dec-2017 07:55:27 PM', 'verifycode': '01', 'checktype': 'Test'}
1811: {'mchdb_pk': 1816, 'sensorid': 1, 'userid': '2232', 'checktime': '10-Dec-2017 08:02:04 AM', 'verifycode': '01', 'checktype': 'Test'}
1812: {'mchdb_pk': 1817, 'sensorid': 1, 'userid': '3071', 'checktime': '09-Dec-2017 08:04:32 AM', 'verifycode': '01', 'checktype': 'Test'}
1813: {'mchdb_pk': 1818, 'sensorid': 1, 'userid': '264', 'checktime': '08-Dec-2017 08:22:26 PM', 'verifycode': '02', 'checktype': 'Test'}
1814: {'mchdb_pk': 1819, 'sensorid': 1, 'userid': '585', 'checktime': '11-Dec-2017 08:34:07 AM', 'verifycode': '02', 'checktype': 'Test'}
1815: {'mchdb_pk': 1820, 'sensorid': 1, 'userid': '249', 'checktime': '09-Dec-2017 07:51:20 PM', 'verifycode': '02', 'checktype': 'Test'}
1816: {'mchdb_pk': 1821, 'sens

In [131]:
for c in data['items']:
    r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att',json = c)

In [94]:
getMaxNum1 = requests.get('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=1') #getting max number from machine 1
getMaxNum2 = requests.get('http://151.80.237.86:1251/ords/demo7/zkt/zkt_att?sensorid=2')#getting max number from machine 2
getMaxNum3 = requests.get('http://151.80.237.86:1251/ords/demo7/zkt/zkt_att?sensorid=3')#getting max number from machine 3

dataJson1 = getMaxNum1.json()
dataJson2= getMaxNum2.json()
dataJson3 = getMaxNum3.json()

#Storing the data  into variable to upload after the last record
for val1 in dataJson1['items']:
    max_value1 = val1['max(mchdb_pk)']

for val2 in dataJson2['items']:
    max_value2 = val2['max(mchdb_pk)'] 

for val3 in dataJson3['items']:
    max_value3 = val3['max(mchdb_pk)'] 

In [95]:
print('Check the records')
display(max_value1)

Check the records


1392

In [96]:
for val in data['items']: #updating the data after the last record to their locations
    if val['mchdb_pk'] >= max_value1 + 1:
        if val['verifycode'] == '01':
            r = requests.post('http://151.80.237.86:1251/ords/zkt/ZktAtt/att?sensorid=1', json = val)
    elif val['mchdb_pk'] >= max_value2 + 1:  #updating the data after the last record to their locations
        if val['verifycode'] == '02':
            r = requests.post('http://151.80.237.86:1251/ords/demo7/zkt/zkt_att?sensorid=2', json = val)
    elif val['mchdb_pk'] >= max_value3 + 1:  #updating the data after the last record to their locations
        if val['verifycode'] == '03':
            r = requests.post('http://151.80.237.86:1251/ords/demo7/zkt/zkt_att?sensorid=3', json = val)

KeyboardInterrupt: 

In [80]:
# Just for check the logic
i = 0
for val in data['items']:
    if val['mchdb_pk'] >= max_value + 1:
        if val['verifycode'] == '01':
            print(val)
            i += 1
            if i %5 == 0:
                break
            
        

{'mchdb_pk': 1642, 'sensorid': 1, 'userid': '265', 'checktime': '09-Dec-2017 08:14:14 AM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 1644, 'sensorid': 1, 'userid': '1689', 'checktime': '07-Dec-2017 08:18:04 AM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 1646, 'sensorid': 1, 'userid': '2217', 'checktime': '10-Dec-2017 07:59:45 PM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 1647, 'sensorid': 1, 'userid': '2400', 'checktime': '09-Dec-2017 08:38:01 AM', 'verifycode': '01', 'checktype': 'Test'}
{'mchdb_pk': 1648, 'sensorid': 1, 'userid': '2587', 'checktime': '07-Dec-2017 09:10:25 AM', 'verifycode': '01', 'checktype': 'Test'}


In [146]:
pwd

'C:\\Users\\Danyal\\Desktop\\New folder'